### Importing required libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as tf
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
import PIL
import cv2
%matplotlib inline

### Setting device as GPU

In [2]:
if torch.cuda.is_available():
    print("GPU available:", torch.cuda.get_device_name())
    device = torch.device('cuda:0')
else:
    print("******Workking on CPU*******")
    device = torch.device('cpu')
device

GPU available: GeForce GTX 1070


device(type='cuda', index=0)

###  Prepare data

In [3]:
CROP_SIZE = 128
class TrainDataset(Dataset):
    def __init__(self, path):
        super(TrainDataset, self).__init__()
        self.image_names = [os.path.join(path,x) for x in os.listdir(path)]
        self.lr_preprocess = tf.Compose([tf.ToPILImage(),tf.Resize((CROP_SIZE,CROP_SIZE), interpolation = PIL.Image.BICUBIC),tf.ToTensor()])
        self.hr_preprocess = tf.Compose([tf.ToTensor()])
        self.len = len(self.image_names)
    def __len__(self):
        return self.len
    def __getitem__(self, index):
        img = self.image_names[index]
        hr = self.hr_preprocess(PIL.Image.open(img))
        lr = self.lr_preprocess(hr)
        return lr,hr

lr_preprocess = tf.Compose([tf.ToPILImage(),tf.Resize((CROP_SIZE,CROP_SIZE), interpolation = PIL.Image.BICUBIC),tf.ToTensor()])
hr_preprocess = tf.Compose([tf.ToTensor()])
img =PIL.Image.open(path)
img_hr = hr_preprocess(PIL.Image.open(path))
img_lr = lr_preprocess(img_hr)

# Model

<img src = "https://miro.medium.com/max/4916/1*zsiBj3IL4ALeLgsCeQ3lyA.png">

### Generator Network

In [16]:
#Residual Block
class Residual_Block(nn.Module):
    def __init__(self):
        super(Residual_Block, self).__init__()
        self.net = nn.Sequential(
        nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(num_features = 64),
        nn.PReLU(),
        nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(num_features = 64)
        )
    def forward(self, x):
        x = x+self.net(x)
        #print("After residial block:",x.shape)
        return x
#Upsample Block
class Upsample_Block(nn.Module):
    def __init__(self, in_ch,scale_factor):
        super(Upsample_Block, self).__init__()
        self.net  = nn.Sequential(
        nn.Conv2d(in_channels = in_ch, out_channels = in_ch*(scale_factor**2), kernel_size = 3, stride = 1, padding = 1),
        nn.PixelShuffle(upscale_factor = scale_factor),
        nn.PReLU()
        )
    def forward(self, x):
        x = self.net(x)
        #print("After upsampling:",x.shape)
        return x
        

#Generator Block
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        #First Block
        self.block1 = nn.Sequential(
        nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 9, padding = 4,stride = 1),
        nn.PReLU()
        )
        #8 Residual Blocks
        self.residuals = nn.ModuleList([Residual_Block() for x in range(8)]) 
        self.block2 = nn.Sequential(
        nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1,padding = 1),
        nn.BatchNorm2d(num_features = 64)
        )
        # 2 Upsampling Blocks
        self.upsamples = nn.Sequential(
        Upsample_Block(64,2),
        Upsample_Block(64,2),
        nn.Conv2d(in_channels = 64, out_channels = 3, kernel_size = 9, stride = 1, padding = 4)
        )
    def forward(self, x):
        #print("Initial shape:",x.shape)
        #print("Device:",device)
        x = self.block1(x)
        #print("After Block 1:", x.shape)
        x_res = self.residuals[0](x)
        for i in range(7):
            x_res = self.residuals[i+1](x_res)
        x_res = self.block2(x)
        x = x+x_res
        x = self.upsamples(x)
        #print("Output Shape:",x.shape)
        return (torch.tanh(x)+1)/2

### Discriminator Network

In [5]:
class Discriminator(nn.Module):
    def __init__(self, l=0.2):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
        nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 3, stride = 1),
        nn.LeakyReLU(l),
        
        nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 2, padding = 1),
        nn.BatchNorm2d(num_features = 64),
        nn.LeakyReLU(l),
            
        nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(num_features = 128),
        nn.LeakyReLU(l),
        
        nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 3, stride = 2, padding = 1),
        nn.BatchNorm2d(num_features = 128),
        nn.LeakyReLU(l),
            
        nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(num_features = 256),
        nn.LeakyReLU(l),
            
        nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 2, padding = 1),
        nn.BatchNorm2d(num_features = 256),
        nn.LeakyReLU(l),
            
        nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(num_features = 512),
        nn.LeakyReLU(l),
            
        nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 2, padding = 1),
        nn.BatchNorm2d(num_features = 512),
        nn.LeakyReLU(l),
            
        nn.AdaptiveAvgPool2d(1),
        nn.Conv2d(512, 1024, kernel_size = 1),
        nn.LeakyReLU(l),
        nn.Conv2d(1024,1,kernel_size = 1)
        )
    def forward(self,x):
        y = self.net(x)
        yhat = torch.sigmoid(y).view(x.shape[0])
        return yhat

Lets test the dimensional correctness with a random vector.

In [6]:
x = torch.randn((5,3,512,512))
disc = Discriminator()
y = disc(x)
y.data.shape

torch.Size([5])

Look Good!

##  Training Procedure

The generic method of training GANs is given bellow. It was proposed by Ian Goodfellow et al in the first paper that introduced GANs.

<img src = "https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/05/Summary-of-the-Generative-Adversarial-Network-Training-Algorithm-1024x669.png" width = 600>

### Pretraining Generator

In [40]:
def pre_train(loader):
    optimizerG = torch.optim.Adam(netG.parameters())
    print("Pretraining Generator:")
    for epoch in range(PRETRAIN_EPOCHS):
        print("\tEpoch: ",epoch)
        netG.train()
        for lr, hr in tqdm(loader):
            lr = lr.cuda()
            hr = hr.cuda()
            sr  = netG(lr)
            netG.zero_grad()
            loss = mse(sr,hr)
            loss.backward()
            optimizerG.step()

### Training GAN

In [71]:
def train_GAN(loader):
    print("Training GAN:")
    for epoch in range(EPOCHS):
        print("\tEpoch: ", epoch)
        netG.train()
        netD.train()
        for lr,hr in tqdm(loader):
            lr = lr.cuda()
            hr = hr.cuda()
            #Train Discriminator
            netD.zero_grad()
            
            logits_real = netD(hr)
            logits_fake = netD(netG(lr).detach())
            
            logits_real = logits_real.cuda()
            logits_fake = logits_fake.cuda()
            #Label Smoothing
            real = torch.randn(logits_real.size())*0.25+0.85
            fake = torch.randn(logits_fake.size())*0.15
            
            real = real.cuda()
            fake = fake.cuda()
            #Label Flipping (not implemented...yet)
            d_loss = bce(logits_real,real) + bce(logits_fake, fake)
            d_loss.backward()
            optimizerD.step()
            
            #Train Generator
            netG.zero_grad()
            sr = netG(lr)
            content_loss = mse(sr,hr)
            
            logits_fake_new = netD(sr)
            adverserial_loss = bce(logits_fake_new, torch.ones_like(logits_fake_new))
            g_loss = content_loss + 1e-2*adverserial_loss
            g_loss.backward()
            optimizerG.step()
                               

In [ ]:
#path = os.path.join(os.getcwd(),"data")
#path = "D:\\Games Setup\\data"


Checking if Dataset object works as expected-

In [ ]:
# for lr,hr in train_set:
#     hr = np.transpose(hr.numpy(),(1,2,0))[:,:,::-1]
#     lr = np.transpose(lr.numpy(),(1,2,0))[:,:,::-1]
#     #print("HR shape:",hr.shape)
#     cv2.imshow("HR Image",hr)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
#     cv2.imshow("LR Image",lr)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

It does, great!

In [85]:
HR_SIZE = 512
LR_SIZE = 128
SCALING_FACTOR = 4
PRETRAIN_EPOCHS = 2
EPOCHS = 1000

I will first write the basic backbone training function, then add other functionalities such as-
<ul>
    <li>Sending model to GPU
    <li>Label Smoothing(well, I did it anyways)</li> 
    <li>Label Flipping</li>
    <li>Saving checkpoints when training</li>
    <li>Visualizing the model with Tensorboard</li>
    <li>Displaying epochwise performance</li>

In [72]:
netG = Generator()
netD = Discriminator()
optimizerD = torch.optim.Adam(netD.parameters())
optimizerG = torch.optim.Adam(netG.parameters())
mse = nn.MSELoss()
bce = nn.BCELoss()
pretrain = True
train_path = "D:\\Games Setup\\FFHQ\\train"
train_set = TrainDataset(train_path)
test_path =  "D:\\Games Setup\\FFHQ\\test"
test_set = TrainDataset(test_path)
train_loader = DataLoader(train_set, batch_size = 10)
test_loader = DataLoader(test_set, batch_size = 100)

In [73]:
netG.cuda()
netD.cuda()
mse.cuda()
bce.cuda()

BCELoss()

### Passing debug data through our functions first.

In [74]:
import timeit
import time

#### Timing pretrainer

In [77]:
debug_path = "D:\\Games Setup\\data"
debug_set = TrainDataset(debug_path)
debug_loader = DataLoader(debug_set)

In [84]:
start_time = time.time()
pre_train(debug_loader)
end_time = time.time()
print("Time to pretrain 1 image for ",PRETRAIN_EPOCHS, "= ",(end_time-start_time)*100,"ms")

Pretraining Generator:
	Epoch:  0



	Epoch:  1



Time to pretrain 1 image for  2 =  52.59981155395508 ms


####  Timing GAN trainer

In [88]:
start_time = time.time()
train_GAN(debug_loader)
end_time = time.time()
print("Time to train GAN on 1 image for ",EPOCHS, " epochs = ",(end_time-start_time),"s")

Training GAN:
	Epoch:  0



	Epoch:  1



	Epoch:  2



	Epoch:  3



	Epoch:  4


...


	Epoch:  996



	Epoch:  997



	Epoch:  998



	Epoch:  999



Time to train GAN on 1 image for  1000  epochs =  446.95525336265564 s


Time to train GAN on 1 image for  1000  epochs =  446.95525336265564 s

Holy fuck! Training 1 image takes 7 minutes. Training 50,000 images would take 250 days. Bloody Hell!